# 1. Imported dependencies

In [1]:
#Importing Dependencies
import pandas as pd
import numpy as np
import requests
import json
import time
import datetime
from config import tmdb_key
from pprint import pprint

# 2. Source 1. CSV

In [2]:
ntfl = pd.read_csv('source_data/NetflixOriginals1.csv', encoding = "ISO-8859-1")
ntfl.head(3)

,Title,Genre,Premiere,Runtime,IMDB Score,Language
0,Enter the Anime,Documentary,"August 5, 2019",58,2.5,English/Japanese
1,Dark Forces,Thriller,"August 21, 2020",81,2.6,Spanish
2,The App,Science fiction/Drama,"December 26, 2019",79,2.6,Italian


In [3]:
len(ntfl['Title'])

584

In [4]:
ntfl_df = ntfl[["Title","Genre","Runtime","Language"]]
ntfl_df.head(3)

,Title,Genre,Runtime,Language
0,Enter the Anime,Documentary,58,English/Japanese
1,Dark Forces,Thriller,81,Spanish
2,The App,Science fiction/Drama,79,Italian


In [5]:
# full_dates = net_flix['Premiere']
# converted = pd.to_datetime(full_dates)
# net_flix['Premiere'] = converted
# net_flix.head()

In [6]:
genre_sep_df = ntfl_df.set_index(['Title','Runtime', 'Language']).apply(lambda x: x.str.split('/').explode()).reset_index()      
genre_sep_df.head(5)

,Title,Runtime,Language,Genre
0,Enter the Anime,58,English/Japanese,Documentary
1,Dark Forces,81,Spanish,Thriller
2,The App,79,Italian,Science fiction
3,The App,79,Italian,Drama
4,The Open House,94,English,Horror thriller


In [7]:
lang_sep_df = ntfl_df.set_index(['Title','Runtime','Genre']).apply(lambda x: x.str.split('/').explode()).reset_index()
lang_sep_df.head(3)

,Title,Runtime,Genre,Language
0,Enter the Anime,58,Documentary,English
1,Enter the Anime,58,Documentary,Japanese
2,Dark Forces,81,Thriller,Spanish


In [8]:
csv_df1 = lang_sep_df
csv_df1.head()

,Title,Runtime,Genre,Language
0,Enter the Anime,58,Documentary,English
1,Enter the Anime,58,Documentary,Japanese
2,Dark Forces,81,Thriller,Spanish
3,The App,79,Science fiction/Drama,Italian
4,The Open House,94,Horror thriller,English


In [9]:
csv_df1["actors"] = " "
csv_df1.head(2)

,Title,Runtime,Genre,Language,actors
0,Enter the Anime,58,Documentary,English,
1,Enter the Anime,58,Documentary,Japanese,


In [10]:
csv_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     610 non-null    object
 1   Runtime   610 non-null    int64 
 2   Genre     610 non-null    object
 3   Language  610 non-null    object
 4   actors    610 non-null    object
dtypes: int64(1), object(4)
memory usage: 24.0+ KB


In [11]:
csv_df1["actors"].fillna(130, inplace = True)
csv_df1

,Title,Runtime,Genre,Language,actors
0,Enter the Anime,58,Documentary,English,
1,Enter the Anime,58,Documentary,Japanese,
2,Dark Forces,81,Thriller,Spanish,
3,The App,79,Science fiction/Drama,Italian,
4,The Open House,94,Horror thriller,English,
...,...,...,...,...,...
605,Winter on Fire: Ukraine's Fight for Freedom,91,Documentary,Ukranian,
606,Winter on Fire: Ukraine's Fight for Freedom,91,Documentary,Russian,
607,Springsteen on Broadway,153,One-man show,English,
608,Emicida: AmarElo - It's All For Yesterday,89,Documentary,Portuguese,


In [12]:
csv_df1 = csv_df1.rename(columns = {'Title':'title','Runtime':'runtime',"Genre":"genre","Language":"languages"}) 
csv_df1

,title,runtime,genre,languages,actors
0,Enter the Anime,58,Documentary,English,
1,Enter the Anime,58,Documentary,Japanese,
2,Dark Forces,81,Thriller,Spanish,
3,The App,79,Science fiction/Drama,Italian,
4,The Open House,94,Horror thriller,English,
...,...,...,...,...,...
605,Winter on Fire: Ukraine's Fight for Freedom,91,Documentary,Ukranian,
606,Winter on Fire: Ukraine's Fight for Freedom,91,Documentary,Russian,
607,Springsteen on Broadway,153,One-man show,English,
608,Emicida: AmarElo - It's All For Yesterday,89,Documentary,Portuguese,


In [13]:
csv_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      610 non-null    object
 1   runtime    610 non-null    int64 
 2   genre      610 non-null    object
 3   languages  610 non-null    object
 4   actors     610 non-null    object
dtypes: int64(1), object(4)
memory usage: 24.0+ KB


# 3. Source2 JSON

In [14]:
json_df2 = pd.read_json('source_data/movie.json')
json_df2.head(1)

,title,rating,year,users_rating,votes,metascore,img_url,countries,languages,actors,genre,tagline,description,directors,runtime,imdb_url
0,A Rainy Day in New York,PG-13,2019.0,6.6,"21,903",44.0,https://m.media-amazon.com/images/M/MV5BODAwZD...,[USA],[English],"[Timothée Chalamet, Elle Fanning, Liev Schreib...","[Comedy, Romance]",None,A young couple arrives in New York for a weeke...,[Woody Allen],92 min,https://www.imdb.com/title/tt7139936/


In [15]:
json_df2.columns

Index(['title', 'rating', 'year', 'users_rating', 'votes', 'metascore',
       'img_url', 'countries', 'languages', 'actors', 'genre', 'tagline',
       'description', 'directors', 'runtime', 'imdb_url'],
      dtype='object')

In [16]:
print(len(csv_df1['title']))
print(len(csv_df1['runtime']))
print(len(json_df2['title']))
print(len(json_df2['runtime']))

610
610
62058
62058


In [17]:
ext_movie_df = json_df2[["title","runtime", "genre","languages","actors"]]
ext_movie_df.head(2)

,title,runtime,genre,languages,actors
0,A Rainy Day in New York,92 min,"[Comedy, Romance]",[English],"[Timothée Chalamet, Elle Fanning, Liev Schreib..."
1,Murder Manual,91 min,"[Horror, Thriller]",[English],"[Emilia Clarke, Hadley Fraser, Sylvia Panacion..."


# 4. Merged CSV and JSON.

In [18]:
# 2 different source is concateneated based on the same columns heads 
movie_df = pd.concat([ext_movie_df,csv_df1], ignore_index=True)
movie_df.head(3)

,title,runtime,genre,languages,actors
0,A Rainy Day in New York,92 min,"[Comedy, Romance]",[English],"[Timothée Chalamet, Elle Fanning, Liev Schreib..."
1,Murder Manual,91 min,"[Horror, Thriller]",[English],"[Emilia Clarke, Hadley Fraser, Sylvia Panacion..."
2,Ferris Bueller's Day Off,103 min,[Comedy],"[English, German]","[Matthew Broderick, Alan Ruck, Mia Sara, Jeffr..."


In [19]:
print(len(movie_df['title']))
print(len(movie_df['runtime']))

62668
62668


In [20]:
# From here unpack json file that has rows as a [] or [[0],[1],[2],...[n]] due to normalize df for RDB
json_df=movie_df

In [21]:
titles_to_list = json_df['title'].tolist()
#titles_to_list
actors_l_of_lists = json_df['actors'].tolist()
actors_l_of_lists
genres_l_of_lists = json_df['genre'].tolist()
genres_l_of_lists
langs_l_of_lists = json_df['languages'].tolist()
runtimes_l_of_lists = json_df['runtime'].tolist()

In [22]:
movie_df[['run_t','mins']] = movie_df.runtime.str.split(expand=True)
movie_df.head(2)

,title,runtime,genre,languages,actors,run_t,mins
0,A Rainy Day in New York,92 min,"[Comedy, Romance]",[English],"[Timothée Chalamet, Elle Fanning, Liev Schreib...",92,min
1,Murder Manual,91 min,"[Horror, Thriller]",[English],"[Emilia Clarke, Hadley Fraser, Sylvia Panacion...",91,min


In [23]:
new_movie_df=movie_df[["title","run_t", "languages", "genre"]]
new_movie_df.head(2)

,title,run_t,languages,genre
0,A Rainy Day in New York,92,[English],"[Comedy, Romance]"
1,Murder Manual,91,[English],"[Horror, Thriller]"


In [24]:
#title & actors normalized tables for each columns 
counter=0
titles_to_expand_match_actors = []
for actors_l in actors_l_of_lists:
    for title in range(0,len(actors_l)):
        titles_to_expand_match_actors.append(titles_to_list[counter])
    counter += 1
#titles_to_expand_match_actors

actors_unpacked_to_outer_list = []
for actors_l in actors_l_of_lists:
    #print(actors_list)
    for actors in actors_l:
        actors_unpacked_to_outer_list.append(actors)
#actors_unpacked_to_outer_list

In [ ]:
actors_associated_with_titles_df = pd.DataFrame({'Title': titles_to_expand_match_actors,
                                                 'Performer': actors_unpacked_to_outer_list
                                                }
                                               )
actors_associated_with_titles_df.head()

In [ ]:
actors_associated_with_titles_df.info()

In [ ]:
actors_associated_with_titles_df.to_csv('actors1.csv', index = False)
pd.read_csv('actors1.csv')

In [ ]:
#title & genres
counter=0
titles_to_expand_match_genres = []
for genre_l in genres_l_of_lists:
    for title in range(0,len(genre_l)):
        titles_to_expand_match_genres.append(titles_to_list[counter])
    counter += 1
# movie titles are expanded to match with different actors played in the movies   
#titles_to_expand_match_genres

genres_unpacked_to_outer_list = []
for genres_l in genres_l_of_lists:
    #print(actors_list)
    for genres in genres_l:
        genres_unpacked_to_outer_list.append(genres)
#genres_unpacked_to_outer_list
#titles_to_expand_match_genres

In [ ]:
genres_associated_with_titles_df = pd.DataFrame({'Title': titles_to_expand_match_genres,
                                                 'Genre': genres_unpacked_to_outer_list
                                                }
                                               )
genres_associated_with_titles_df.head(5)

In [ ]:
#genres_associated_with_titles_df.to_csv('genres1.csv', index = False)
# pd.read_csv('genres1.csv')

In [ ]:
#title & language
counter=0
titles_to_expand_match_langs = []
for lang_l in langs_l_of_lists:
    for title in range(0,len(lang_l)):
        titles_to_expand_match_langs.append(titles_to_list[counter])
    counter += 1
# movie titles are expanded to match with different actors played in the movies   
titles_to_expand_match_langs

langs_unpacked_to_outer_list = []
for langs_l in langs_l_of_lists:
    #print(actors_list)
    for langs in langs_l:
        langs_unpacked_to_outer_list.append(langs)
#langs_unpacked_to_outer_list

In [ ]:
langs_associated_with_titles_df = pd.DataFrame({'Title': titles_to_expand_match_langs,
                                                 'Language': langs_unpacked_to_outer_list
                                                }
                                               )
langs_associated_with_titles_df.head(5)

In [ ]:
#langs_associated_with_titles_df.to_csv('languages1.csv', index = False)
#pd.read_csv('languages1.csv')

# 5. Cleaning, and preparing tables for ERD 
1. Creating Nflx_movie table PK unique title_id as pk used table with  
   columns such as since Title and run_t are not duplicated to match row vals.

In [ ]:
run_t_df = movie_df[["title","run_t"]]
run_t_df

# 6. Unique Movie_titles for table, PK movie_id

In [ ]:
runtime_table = run_t_df.reset_index().rename(columns = {"index": "movie_id"})
runtime_table

In [ ]:
# on movie title can be many titles 
unique_movie = runtime_table["title"].value_counts()
unique_movie

In [ ]:
# unique_m_title.to_csv('languages1.csv', index = False)

In [ ]:
runtime_table = unique_movie.reset_index()
runtime_table.head(2)

In [ ]:
movie_table = runtime_table.reset_index().rename(columns ={"level_0":"title_id"})
movie_table.head(2)

In [ ]:
nflx_movie = movie_table.rename(columns = {"index":"title","title":"numb_of_versions"})
nflx_movie.head(2)

In [ ]:
nflx_movie = nflx_movie[["title_id","title","numb_of_versions"]]
nflx_movie.head(3)

In [ ]:
nflx_movie.to_csv('sql_data/nflx_movie.csv1', index = False)

In [ ]:
#pd.read_csv('Nflx_movie.csv')

## 7. Table 2  for ERD nflx_m_runtime

In [ ]:
run_t_df = movie_df[["title","run_t"]]
run_t_df

In [ ]:
run_t_clean = run_t_df.fillna(0)
run_t_clean

In [ ]:
print(run_t_clean.dtypes)


In [ ]:
runtime_table = run_t_clean.reset_index().rename(columns = {"index": "r_time_id"})
runtime_table

In [ ]:
runtime_table.info()

In [ ]:
#runtime_table["run_t"] = pd.to_numeric(runtime_table["run_t"])
runtime_table.drop(runtime_table.index[18957])

In [ ]:
print(runtime_table.dtypes)

In [ ]:
runtime_table['run_t'] = pd.to_numeric(runtime_table['run_t'], errors = 'coerce')
# df['mix_col'] = pd.to_numeric(df['mix_col'], errors='coerce')

In [ ]:
runtime_table

In [ ]:
print(runtime_table.dtypes)

In [ ]:
unique_time = runtime_table["run_t"].value_counts()
unique_time

In [ ]:
u_runtime_table = unique_time.reset_index()
u_runtime_table.head(5)

In [ ]:
u_rt_table = u_runtime_table.reset_index().rename(columns ={"level_0":"rt_id"})
u_rt_table.head(2)

In [ ]:
nflx_rt = u_rt_table.rename(columns = {"index":"r_time","run_t":"occurrences"})
nflx_rt.head(2)

In [ ]:
nflx_m_runtime = nflx_rt[["rt_id","r_time","occurrences"]]
nflx_m_runtime.head(3)

In [ ]:
nflx_m_runtime.to_csv('sql_data/nflx_m_runtime.csv', index = False)

In [ ]:
print(nflx_m_runtime.dtypes)

# 8. Table 3 nflx_actors for ERD  unque actors

In [ ]:
actors_table = actors_associated_with_titles_df
actors_table.head(2)

In [ ]:
# actors_t = run_t_df.reset_index().rename(columns = {"index": "movie_id"})
a_t = actors_table.reset_index()
a_t

In [ ]:
# actors_t = run_t_df.reset_index().rename(columns = {"index": "movie_id"})
a_t = actors_table.reset_index()
a_t

In [ ]:
perf_table = a_t.rename(columns= {"index":"actor_id"})
perf_table 

In [ ]:
uniq_perf = perf_table["Performer"].value_counts()
uniq_perf

In [ ]:
to_set_ind = uniq_perf.reset_index()
to_set_ind

In [ ]:
actor_t = to_set_ind.reset_index().rename(columns ={"level_0":"actor_id"})
actor_t

In [ ]:
nflx_actors_b = actor_t.rename(columns = {"index":"actors","Performer":"appearance"})
nflx_actors_b

In [ ]:
nflx_actors = nflx_actors_b.drop([0])

In [ ]:
nflx_actors.to_csv("sql_data/nflx_actors.csv", index = False)
# nflx_m_runtime.to_csv('sql_data/nflx_m_runtime.csv', index = False)

In [ ]:
print(nflx_actors.dtypes)

# 9. Table 4. Language table 

In [ ]:
language_table = langs_associated_with_titles_df
language_table.head(2)

In [ ]:
language_table.info()

In [ ]:
l_t = language_table.reset_index()
l_t

In [ ]:
lang_table = l_t.rename(columns= {"index":"lang_id"})
lang_table

In [ ]:
uniq_lang = lang_table["Language"].value_counts()
uniq_lang

In [ ]:
to_set_idx = uniq_lang.reset_index()
to_set_idx

In [ ]:
lang_t = to_set_idx.reset_index().rename(columns ={"level_0":"lang_id"})
lang_t

In [ ]:
nflx_langs = lang_t.rename(columns = {"index":"languages", "Language":"frequency"})
nflx_langs

In [ ]:
nflx_languages = nflx_langs[["lang_id","languages","frequency"]]
nflx_languages

In [ ]:
nflx_languages.to_csv('sql_data/nflx_languages.csv', index = False)

In [ ]:
print(nflx_languages.dtypes)

# 10. Table5 nflx_genre for ERD

In [ ]:
genre_table = genres_associated_with_titles_df
genre_table.head()

In [ ]:
g_t = genre_table.reset_index()
g_t

In [ ]:
genre_table = g_t.rename(columns= {"index":"genre_id"})
genre_table

In [ ]:
genre_table.info()

In [ ]:
uniq_genre = genre_table["Genre"].value_counts()
uniq_genre

In [ ]:
to_set_idx = uniq_genre.reset_index()
to_set_idx

In [ ]:
genre_t = to_set_idx.reset_index().rename(columns ={"level_0":"genre_id"})
genre_t

In [ ]:
nflx_genres = genre_t.rename(columns = {"index":"genre","Genre":"freq"})
nflx_genres

In [ ]:
nflx_genres.to_csv("sql_data/nflx_genres.csv", index = False)

In [ ]:
print(nflx_genres.dtypes)

# 11. Table6. Genre Table for ERD 

In [ ]:
genre_table = genres_associated_with_titles_df
genre_table

In [ ]:
genre_table.to_csv("sql_data/genre_table1.csv", index = False)

# 12. Table 7 for ERD

In [ ]:
language_table = langs_associated_with_titles_df
language_table

In [ ]:
language_table.to_csv("sql_data/language_table.csv", index = False)

# 13. Table8  for ERD

In [ ]:
runtime_table

In [ ]:
runtime_table_cln = runtime_table.dropna()
runtime_table_cln

In [ ]:
runtime_table.to_csv("sql_data/runtime_table_cln.csv", index = False)

# 14. Table9 for ERD

In [ ]:
actors_table = actors_associated_with_titles_df
actors_table.head(2)

In [ ]:
actors_table.to_csv("sql_data/runtime_table_cln.csv", index = False)

# 15. Fixed delimiter errors Importing tables into PostGress

In [ ]:
#cln_csv = pd.read_csv('sql_data/nflx_movie1.csv')

In [ ]:
#cln_csv.tail(10)

In [ ]:
cln_csv.columns

In [ ]:
result = cln_csv.iloc[[18262],[1]]
result = cln_csv.iloc[[25427],[1]]
result = cln_csv.iloc[[43625],[1]]
result

In [ ]:
cln_csv.loc[(cln_csv.title == "Feudin', Fussin' and A-Fightin'"),"title"]="Feudin Fussin and A-Fightin"
cln_csv.loc[(cln_csv.title == "God Said, 'Ha!'"),"title"]="God Said. 'Ha!'"
cln_csv.loc[(cln_csv.title == "'Sheba, Baby'"),"title"]="'Sheba Baby'"

cln_csv

In [ ]:
cln_csv.to_csv("sql_data/nflx_movie.csv", index = False)


In [ ]:
title_id=nflx_movie["title_id"]
title_id